In [2]:
# Deliverable 2: Determine Bias of Vine Reviews
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.0'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

'apt-get' is not recognized as an internal or external command,
operable program or batch file.
The system cannot find the path specified.
'wget' is not recognized as an internal or external command,
operable program or batch file.
tar: Error opening archive: Failed to open '$SPARK_VERSION-bin-hadoop3.tgz'


Exception: Unable to find py4j in /content/spark-3.3.0-bin-hadoop3\python, your SPARK_HOME may not be configured correctly

In [ ]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("DataFrameBasics").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Digital_Video_Games_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

In [ ]:
df.count()

In [ ]:
# Load in a sql function to use columns
from pyspark.sql.functions import col

# Filter for only columns with total_votes equal or greater than 20
votes20plus_df = df.filter(col("total_votes") >= 20)
votes20plus_df.show(5)

In [ ]:
votes20plus_df.count()

In [ ]:
# Filter the data for helpful_votes/total_votes is >= 50%
vote50plus_df = votes20plus_df.filter(col("helpful_votes")/col("total_votes") >= .50)
vote50plus_df.show(5)

In [ ]:
vote50plus_df.count()

In [ ]:
# Filter for data where a review was written as part of the Vine program
paid_vine_df = vote50plus_df.filter(col("vine") == 'Y')
paid_vine_df.show()

In [ ]:
paid_vine_df.count()

In [ ]:
# Filter for data where a review was not part of the Vine program
unpaid_vine_df = vote50plus_df.filter(col("vine") == 'N')
unpaid_vine_df.show(5)

In [ ]:
# Finding total # of review paid
paid_vine_df.count()

In [ ]:
# Finding total # of 5-star reviews paid
paid_star_rating_df = paid_vine_df.filter(col("star_rating") == 5)
paid_star_rating_df.count()

In [ ]:
# Finding percentage of 5-star reviews paid

# percent_paid = (paid_star_rating_df.count()/paid_vine_df.count())*100
# print("Percentage of 5-star reviews for paid Vine program was: %f" % percent_paid + "%")

#### Unfortantly, my dataset did not have Y's in the vine column.  
#### This code will result in a divsion by zero error. There are 0% paid Vine 5-star reviews (or any other rating level.)

In [ ]:
# Finding total # of review unpaid
unpaid_vine_df.count()

In [ ]:
# Finding total # of 5-star reviews unpaid
unpaid_star_rating_df = unpaid_vine_df.filter(col("star_rating") == 5)
unpaid_star_rating_df.count()

In [ ]:
# Finding percentage of 5-star reviews unpaid
percent_unpaid = (unpaid_star_rating_df.count()/unpaid_vine_df.count())*100
print("Percentage of 5-star reviews for unpaid Vine program was: %f" % percent_unpaid + "%")